<a href="https://colab.research.google.com/github/karenbennis/BigData/blob/master/ChallengeETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Download a Postgres driver that will allow Spark to interact with Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-06-13 20:05:26--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.04MB/s    in 0.8s    

2020-06-13 20:05:28 (1.04 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ChallengeETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in user_data from S3 Buckets
from pyspark import SparkFiles
url = "https://karenbennis-bucket.s3.us-east-2.amazonaws.com/Datasets/amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
amazon_reviews_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
amazon_reviews_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|     179886| RY01SAV7HZ8QO|B00NTI0CQ2|     667358431|Aketek 1080P LED ...|Home Entertainment|          4|            0|          0|   N|                Y|good enough for m...|not the best pict...|2015-08-31 00:00:00|
|         US|   37293769|R1XX8SDGJ4MZ4L|B00BUCLVZU|     621695622|TiVo Mini with

In [ ]:
# Count number of rows
row_number = amazon_reviews_df.count()
row_number

705889

In [ ]:
# Drop null values
dropna_df = amazon_reviews_df.dropna()
dropna_df.show()
# Count number of rows
row_number = dropna_df.count()
row_number

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|     179886| RY01SAV7HZ8QO|B00NTI0CQ2|     667358431|Aketek 1080P LED ...|Home Entertainment|          4|            0|          0|   N|                Y|good enough for m...|not the best pict...|2015-08-31 00:00:00|
|         US|   37293769|R1XX8SDGJ4MZ4L|B00BUCLVZU|     621695622|TiVo Mini with

705850

In [ ]:
# Get count of customer_id
customer_df = dropna_df.groupBy("customer_id").count()
customer_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   10142992|    1|
|   16457323|    6|
|   11935383|    1|
|   46277736|    1|
|   13671072|    1|
|   21453814|    1|
|   17684885|    1|
|   20415768|    1|
|   15212710|    1|
|    5220924|    1|
|   46253451|    6|
|     971908|    1|
|   32829933|    1|
|   51221518|    1|
|   12002637|    2|
|   16105308|    1|
|     135867|    1|
|   47425808|    1|
|   43138273|    1|
|   16411995|    1|
+-----------+-----+
only showing top 20 rows



In [ ]:
# Create customer dataframe to match customer table
customer_df = customer_df.withColumnRenamed(existing='count', new='customer_count')
customer_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   10142992|             1|
|   16457323|             6|
|   11935383|             1|
|   46277736|             1|
|   13671072|             1|
|   21453814|             1|
|   17684885|             1|
|   20415768|             1|
|   15212710|             1|
|    5220924|             1|
|   46253451|             6|
|     971908|             1|
|   32829933|             1|
|   51221518|             1|
|   12002637|             2|
|   16105308|             1|
|     135867|             1|
|   47425808|             1|
|   43138273|             1|
|   16411995|             1|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# Create review_id dataframe to match review_id table
review_id_df = dropna_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
| RY01SAV7HZ8QO|     179886|B00NTI0CQ2|     667358431|2015-08-31 00:00:00|
|R1XX8SDGJ4MZ4L|   37293769|B00BUCLVZU|     621695622|2015-08-31 00:00:00|
|R149Q3B5L33NN5|    8332121|B00RBX9D5W|     143071132|2015-08-31 00:00:00|
|R2ZVD69Z6KPJ4O|   47054962|B00UJ3IULO|     567816707|2015-08-31 00:00:00|
|R1DIKG2G33ZLNP|   23413911|B0037UCTXG|     909557698|2015-08-31 00:00:00|
|R3L6FGKAW0EYFI|    4417771|B004N866SU|     414565179|2015-08-31 00:00:00|
| RAO0QZH5VC6VI|   47900707|B00JE6AOJS|     798450889|2015-08-31 00:00:00|
|R25IK0UAHWNB22|   34112894|B00COL0B7A|     777554234|2015-08-31 00:00:00|
|R2A9IHKZMTMAL1|   20691979|B00QHLSKOE|     885228855|2015-08-31 00:00:00|
| R5XVKTHL6SITI|   25983343|B00UNL2MUW|     164482798|2015-08-31 00:00:00|
|R2QZZOSTDDY1IE|   358160

In [ ]:
# Create products dataframe to match products table
products_df = dropna_df.select(["product_id", "product_title"])

# Drop duplicates of 'product_id'
products_df = products_df.dropDuplicates(['product_id'])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|9967354976|Professional Retr...|
|B00000J025|RCA VR552 4-Head VCR|
|B00004YK3Q|Sylvania DVL500A ...|
|B00005NGRU|Toshiba 50H81 50-...|
|B00005U231|Sony KV36XBR450 3...|
|B000065UTY|Panasonic CT-27SX...|
|B00006CFLW|Hitachi 57XWX20B ...|
|B000093URS|Toshiba 65H93 65-...|
|B0000BZH5T|RCA 27F530T 27" T...|
|B00021Z98A|Panasonic DVD-LS5...|
|B0002IYMT0|Eveready 455 Carb...|
|B0002KQR3C|Samsung TX-P2675W...|
|B0002ZPTVS|Acoustic Wall Til...|
|B0002ZQGRY|Virgin Mobile Kyo...|
|B0006HJII2|Virtual Fx Conver...|
|B00078XIW2|Pioneer PDP4345HD...|
|B00080FPSE|Sony DVP-NC80V/S ...|
|B0009HGXT6|Lilliput 8" LCD M...|
|B0009KEYTO|Pioneer DVR-233S ...|
|B0009X47O8|Samsung DVD-VR325...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
products_df.count()

44810

In [ ]:
# Create vine dataframe to match vine table
vine_df = dropna_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RY01SAV7HZ8QO|          4|            0|          0|   N|
|R1XX8SDGJ4MZ4L|          5|            0|          0|   N|
|R149Q3B5L33NN5|          5|            0|          0|   N|
|R2ZVD69Z6KPJ4O|          1|            0|          2|   N|
|R1DIKG2G33ZLNP|          4|            0|          0|   N|
|R3L6FGKAW0EYFI|          1|            1|          1|   N|
| RAO0QZH5VC6VI|          1|            0|          0|   N|
|R25IK0UAHWNB22|          3|            0|          0|   N|
|R2A9IHKZMTMAL1|          5|            1|          2|   N|
| R5XVKTHL6SITI|          5|            0|          0|   N|
|R2QZZOSTDDY1IE|          3|            3|          6|   N|
|R38CUDCFPSNYTD|          5|            0|          0|   N|
| RM6ZR6NH052YH|          3|            1|          2|   N|
| RUQK5N4WH8UN8|          5|            

In [ ]:
vine_df.count()

705850

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.cigpctobxd5d.us-east-2.rds.amazonaws.com:5432/"
config = {"user":"postgres", 
          "password": "jack1257", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to customer table in RDS
customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write DataFrame to review_id table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write DataFrame to products table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write DataFrame to vine table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

See Vine_table_anlaysis.ipynb for analysis on the vine_table